# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [ ]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
cf.go_offline()
import xlrd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_excel('../data/Online Retail.xlsx')
df = pd.DataFrame(data)
df.head()

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [ ]:
df1 = df[(df['Country'] != 'United Kingdom') & (df.InvoiceDate > '2011-04-01') & (df.InvoiceDate < '2011-04-30')]
df1.head()

In [ ]:
df2 = df1.groupby(["Country"])[(["Quantity", "Revenue"])].sum()
display(df2)

In [ ]:
df2.iplot(kind='bar')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [ ]:
df2 = df[(df['Country'] == 'France') & (df.InvoiceDate > '2011-01-01') & (df.InvoiceDate < '2011-05-31')]
df2.head()

In [ ]:
df3 = df2[(["Quantity", "Revenue"])]
df3.head()

In [ ]:
df3.iplot(kind="lines")

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [ ]:
df4 = df[(df['Description'] == 'PARTY BUNTING')]
df4grouped = df4.groupby('Country').mean()

In [ ]:
party = px.scatter(df4grouped, x=df4grouped["Quantity"], y=df4grouped["UnitPrice"], size = "Quantity", color =df4grouped.index, hover_data=['Revenue'])
party.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [ ]:
countries =  ['France', 'EIRE', 'Germany', 'France', 'Netherlands']

df5 = df[df['Country'].isin(countries)]
df5.head()

In [ ]:
tabla = pd.pivot_table(df5, values='Quantity', index='InvoiceNo', columns='Country', aggfunc='sum')

tabla.iplot(subplots=True, kind='histogram', xTitle='InvoiceNo', yTitle='Quantity')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [ ]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [ ]:
dfrev = df[df['Country'].isin(country_list)]
dfrev = df[df['Description'].isin(product_list)]
dfrev = dfrev.pivot_table(values='Revenue', columns=['Description'], index=['Country'], aggfunc='sum')
dfrev.head()

In [ ]:
fig5 = go.Figure(data=[
    go.Bar(name="CREAM HANGING HEART T-LIGHT HOLDER", x=dfrev.index, y=dfrev["CREAM HANGING HEART T-LIGHT HOLDER"]),
    go.Bar(name='JUMBO BAG RED RETROSPOT', x=dfrev.index, y=dfrev["JUMBO BAG RED RETROSPOT"]),
    go.Bar(name='REGENCY CAKESTAND 3 TIER', x=dfrev.index, y=dfrev["REGENCY CAKESTAND 3 TIER"])])

In [ ]:
fig5.show()

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [ ]:
datauk = uk.groupby(['Day','Month','Year']).sum()
datauk = datauk.reset_index()
datauk.head()

In [ ]:
fig6 = px.line(datauk, x='Day', y="Quantity")
fig6.show()
#??

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

In [ ]:
fig7 = px.scatter(products, x="InvoiceNo", y="CustomerID")
fig7.show()

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.